# FACETpy Full Correction Workflow (Pipe Operator Syntax)

This tutorial runs the full correction workflow using `ProcessingContext | Processor` chaining.

The pipe style is useful in notebooks because each stage can be inspected and modified interactively.

In [ ]:
from pathlib import Path

from facet import (
    AASCorrection,
    CutAcquisitionWindow,
    DownSample,
    EDFExporter,
    HighPassFilter,
    LowPassFilter,
    MedianArtifactCalculator,
    MetricsReport,
    RMSCalculator,
    RMSResidualCalculator,
    RawPlotter,
    SNRCalculator,
    TriggerAligner,
    TriggerDetector,
    UpSample,
    PasteAcquisitionWindow,
    load,
)

try:
    from facet import PCACorrection
except ImportError:
    PCACorrection = None

project_root = Path.cwd()
if not (project_root / "examples").exists() and (project_root.parent / "examples").exists():
    project_root = project_root.parent

input_file = project_root / "examples" / "datasets" / "NiazyFMRI.edf"
output_dir = project_root / "output" / "notebooks"
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / "corrected_pipe_operator.edf"
plot_file = output_dir / "pipe_operator_before_after.png"

print(f"Input file: {input_file}")
assert input_file.exists(), f"Missing dataset: {input_file}"
print(f"PCACorrection available: {PCACorrection is not None}")

## Step 1: Load Recording into a ProcessingContext

`load(...)` creates the initial context object that will be passed through all later pipe operations.

In [ ]:
ctx = load(str(input_file), preload=True, artifact_to_trigger_offset=-0.005)
print(ctx)

## Step 2: Detect Triggers and Prepare for Correction

This stage detects MR triggers, cuts acquisition windows, removes low-frequency drift, upsamples for timing precision, and aligns trigger positions.

In [ ]:
ctx = (
    ctx
    | TriggerDetector(regex=r"\\b1\\b")
    | CutAcquisitionWindow()
    | HighPassFilter(freq=1.0)
    | UpSample(factor=10)
    | TriggerAligner(ref_trigger_index=0, upsample_for_alignment=False)
)

n_triggers = len(ctx.get_triggers()) if ctx.get_triggers() is not None else 0
print(f"Detected triggers: {n_triggers}")

## Step 3: Apply Main Artifact Correction

AAS (`Averaged Artifact Subtraction`) is the main correction method. If available, PCA is applied after AAS to remove structured residual artifact components.

In [ ]:
ctx = ctx | AASCorrection(window_size=30, correlation_threshold=0.975, realign_after_averaging=True)

if PCACorrection is not None:
    ctx = ctx | PCACorrection(n_components=0.95, hp_freq=1.0)
    print("Applied PCACorrection.")
else:
    print("Skipped PCA (PCACorrection unavailable).")

## Step 4: Reconstruct Final Signal Domain

After correction at high sampling rate, this stage downsamples back, restores cut acquisition windows, and applies low-pass filtering for final cleanup.

In [ ]:
ctx = (
    ctx
    | DownSample(factor=10)
    | PasteAcquisitionWindow()
    | LowPassFilter(freq=70.0)
)

print(ctx)

## Step 5: Compute Quality Metrics and Save Comparison Plot

These processors calculate quality indicators (SNR and RMS-based metrics), print a report, and store a before/after visualization.

In [ ]:
ctx = (
    ctx
    | SNRCalculator()
    | RMSCalculator()
    | RMSResidualCalculator()
    | MedianArtifactCalculator()
    | MetricsReport(name="Pipe Operator Workflow Metrics")
    | RawPlotter(
        mode="matplotlib",
        channel="Fp1",
        start=25.0,
        duration=20.0,
        overlay_original=True,
        show=False,
        auto_close=True,
        save_path=str(plot_file),
        title="Pipe Operator Workflow: Before vs After",
    )
)

## Step 6: Export Corrected Recording

This writes the final corrected EEG to EDF while keeping the updated context available for further analysis.

In [ ]:
ctx = ctx | EDFExporter(path=str(output_file), overwrite=True)
print(f"Exported: {output_file}")

## Step 7: Inspect Final Results

Finally, inspect key metrics, processing history, and file existence checks to verify the workflow completed as expected.

In [ ]:
metrics = ctx.metadata.custom.get("metrics", {})
print("Key metrics:")
for key in ("snr", "rms_ratio", "rms_residual", "median_artifact"):
    if key in metrics:
        print(f"  {key}: {metrics[key]}")

history = ctx.get_history()
print(f"\nRecorded processing steps: {len(history)}")
for step in history[:8]:
    print(f"  - {step.name}")
if len(history) > 8:
    print("  ...")

print(f"\nOutput exists: {output_file.exists()}")
print(f"Plot exists: {plot_file.exists()}")